# Invoke Bedrock model for text generation using zero-shot prompt

🎓 AWS BEDROCK LAB:Invoke Bedrock model for text generation using zero-shot prompt

WHAT YOU'LL LEARN:
- How to connect to AWS Bedrock
- How to send prompts to Amazon Nova AI models
- Difference between non-streaming and streaming responses
- How to handle AI model responses

KEY CONCEPTS:
1. Non-streaming: Get complete response all at once (like downloading a file)
2. Streaming: Get response in real-time chunks (like streaming a video)

📚 PREREQUISITE:
Make sure you have AWS credentials configured and access to Amazon Bedrock service.

## Introduction
You are Swami a Customer Service Manager at AnyCompany and some of your customers are not happy with the customer service and are providing negative feedbacks on the service provided by customer support engineers. Now, you would like to respond to those customers humbly aplogizing for the poor service and regain trust. You need the help of an LLM to generate a bulk of emails for you which are human friendly and personalized to the customer's sentiment from previous email correspondence.

## Setup

🔧 SETUP: Import Libraries and Initialize Bedrock Client

This cell sets up everything we need to work with AWS Bedrock:
- We import necessary libraries (json, boto3, etc.)
- We configure AWS credentials and region settings
- We create a Bedrock client that will communicate with AWS AI models

Think of this as "opening the door" to AWS Bedrock services. The boto3_bedrock 
client is your connection to Amazon's AI models.

💡 Tip: If you get authentication errors, uncomment and fill in the AWS configuration 
lines with your region and credentials.

In [11]:
import json
import os
import sys
import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, print_ww

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: eu-north-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.eu-north-1.amazonaws.com)


## Create the prompt

📝 PROMPT PREPARATION: Create Your Request

Here we're preparing what we want to ask the AI model:
- prompt_data: Contains our instruction (write an email from Swami to John Doe)
- body: The complete request formatted for Amazon Nova models

🔑 Amazon Nova Request Structure:
Nova uses a "messages" format with:
  - "role": "user" (who's asking)
  - "content": [{"text": your_prompt}] (what you're asking)

inferenceConfig controls how the AI responds:
  - max_new_tokens: Maximum length of response (4096 tokens ≈ 3000 words)
  - temperature: 0 means deterministic/consistent responses
  - topP: 0.9 controls response diversity

modelId specifies we're using Amazon Nova Lite version 1.0

In [12]:
# create the prompt
prompt_data = """
Command: Write an email from Swami, Customer Service Manager, to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""

# Nova models use a different request format
body = json.dumps({
    "messages": [
        {
            "role": "user",
            "content": [{"text": prompt_data}]
        }
    ],
    "inferenceConfig": {
        "max_new_tokens": 4096,
        "temperature": 0,
        "topP": 0.9
    }
})

modelId = 'amazon.nova-lite-v1:0'
accept = 'application/json'
contentType = 'application/json'

⚡ NON-STREAMING INVOCATION: Get Complete Response at Once

This cell sends your request and waits for the COMPLETE response:
- invoke_model() sends the request and waits for full response
- We parse the JSON response to extract the AI's text

📊 Nova Response Structure:
response_body → output → message → content[0] → text

This is like sending an email and waiting for the complete reply before reading it.

✅ Use this when: You want the entire response before processing it
❌ Don't use when: Generating very long responses where you want to see progress

Error handling catches AccessDeniedException if you don't have proper AWS permissions.

In [13]:
outputText = "\n"

try:
    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    
    # Nova response format
    outputText = response_body.get('output', {}).get('message', {}).get('content', [{}])[0].get('text', '')
    
    print('\t\t\x1b[31m**Non-streaming Output**\x1b[0m\n')
    print(outputText)
    
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubleshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

		**Non-streaming Output**

Subject: Addressing Your Recent Feedback and Our Commitment to Improvement

Dear John Doe,

I hope this email finds you well.

First and foremost, I want to express my sincere gratitude for taking the time to share your feedback regarding the recent service provided by our customer support engineer. At [Company Name], we highly value our customers' opinions and experiences, as they are crucial to our continuous improvement and service excellence.

I understand that the experience you had did not meet your expectations, and for that, I sincerely apologize. It is deeply concerning to hear about any issues that may have caused you inconvenience or frustration. Please rest assured that we take your feedback very seriously and are committed to addressing the concerns you have raised.

To better understand the situation and to ensure that we can provide you with the best possible resolution, I would appreciate it if you could provide more details about the specifi

🌊 STREAMING INVOCATION: Get Response Piece by Piece

This cell uses streaming to receive the AI's response in real-time chunks:
- invoke_model_with_response_stream() starts receiving immediately
- Each "chunk" is a piece of the response as it's generated
- We print each chunk as it arrives

🎯 How Streaming Works:
1. Send request
2. AI starts generating text
3. Receive chunk 1 → print it
4. Receive chunk 2 → print it
5. Continue until complete

Nova Streaming Format:
Each chunk contains: contentBlockDelta → delta → text

✅ Use this when: You want immediate feedback and live updates
📱 Perfect for: Chat interfaces, long responses, better user experience

This is like watching someone type a message in real-time rather than waiting 
for them to finish before seeing anything.

In [14]:
output = []

try:
    response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                
                # Nova streaming format
                if 'contentBlockDelta' in chunk_obj:
                    text = chunk_obj['contentBlockDelta']['delta']['text']
                    output.append(text)
                    print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                    i += 1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubleshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

		**Chunk 1**
Subject

		**Chunk 2**
: Addressing

		**Chunk 3**
 Your

		**Chunk 4**
 Recent

		**Chunk 5**
 Experience with Our Customer Support



		**Chunk 6**
Dear John

		**Chunk 7**
 Doe,

I hope this email

		**Chunk 8**
 finds you well. I

		**Chunk 9**
 am writing to you

		**Chunk 10**
 personally

		**Chunk 11**
 as

		**Chunk 12**
 the

		**Chunk 13**
 Customer Service Manager, Swami

		**Chunk 14**
, in response to the

		**Chunk 15**
 feedback you recently

		**Chunk 16**
 provided regarding

		**Chunk 17**
 your

		**Chunk 18**
 experience with our customer support

		**Chunk 19**
 engineer

		**Chunk 20**
. I

		**Chunk 21**
 sincerely

		**Chunk 22**
 apologize for any inconvenience or frustration

		**Chunk 23**
 you may

		**Chunk 24**
 have encountered, and I

		**Chunk 25**
 want to assure you that your

		**Chunk 26**
 concerns are taken

		**Chunk 27**
 very

		**Chunk 28**
 seriously.

We

		**Chunk 29**
 strive

		**Chunk 30**
 to provide the

		**Chunk 31**
 

📋 FINAL OUTPUT: Display Complete Streamed Response

This cell combines all the chunks we received during streaming:
- ''.join(output) combines all text chunks into one complete string
- Prints the full response for easy reading

Why do we need this?
During streaming (Cell 4), chunks are printed separately. This cell shows you 
the final, complete response all together - useful for copying or reviewing 
the entire output.

Think of it like: Cell 4 shows you the movie frame-by-frame, Cell 5 shows you 
the complete film.

In [5]:
# The relevant portion of the response begins after the first newline character
# Below we print the response beginning after the first occurence of '\n'.

email = outputText[outputText.index('\n')+1:]
print_ww(email)

Subject: Apology for the negative experience you had with our customer support engineer

Dear John Doe,

I am writing to express my deepest apologies for the negative experience you had with our customer
support engineer.

We take customer feedback very seriously and I want to assure you that we are taking steps to
prevent such incidents from happening in the future.

Sincerely,
Bob
Customer Service Manager


## Streaming the response

In [15]:
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)

		**COMPLETE OUTPUT**

Subject: Addressing Your Recent Experience with Our Customer Support

Dear John Doe,

I hope this email finds you well. I am writing to you personally as the Customer Service Manager, Swami, in response to the feedback you recently provided regarding your experience with our customer support engineer. I sincerely apologize for any inconvenience or frustration you may have encountered, and I want to assure you that your concerns are taken very seriously.

We strive to provide the highest level of service to all our customers, and it is disappointing to hear that your experience did not meet your expectations. Your feedback is invaluable to us, as it helps us identify areas where we can improve and ensure that all our customers receive the support they deserve.

I have reviewed the details of your case and would like to address the specific issues you mentioned. Please know that we are taking immediate steps to rectify any shortcomings and to prevent similar issues

### Complete output

In [7]:
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)

		**COMPLETE OUTPUT**


Subject: Apology for the negative experience you had with our customer support engineer
Dear John Doe,

I am writing to express my deepest apologies for the negative experience you had with our customer support engineer. The feedback you provided is extremely valuable to us, and we take your comments seriously.

We strive to provide excellent customer service, and it is disappointing to hear that we fell short of your expectations. I would like to assure you that we are taking steps to address the issues you raised.

Firstly, I would like to investigate the matter further. Could you please provide me with your contact information so that I can reach out to you directly?

Secondly, I would like to offer you a complimentary service to make up for the negative experience you had. This could include a free upgrade to your product, a discount on your next purchase, or a free trial of our premium services.

Please rest assured that we are committed to resolving this i